In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from urllib.parse import urljoin

In [2]:
path = ''
db_name = 'github_repos.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    cur.execute('''
        CREATE TABLE IF NOT EXISTS repositories (
            repo_name TEXT PRIMARY KEY,
            language TEXT,
            stars INTEGER
        );
    ''')
    conn.commit()
except sqlite3.Error as e:
    print('DBエラーが発生しました:', e)
finally:
    if 'conn' in locals() and conn:
        conn.close()

def convert_stars_to_int(star_str):

    if not star_str:
        return 0
    s = star_str.lower().replace(',', '').strip()
    try:
        if 'k' in s:
            return int(float(s.replace('k', '')) * 1000)
        return int(float(s))
    except Exception:
        return 0

def scrape_github_repos(organization_name):
    session = requests.Session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'Accept-Language': 'ja-JP,ja;q=0.9,en-US;q=0.8'
    }

    page = 1
    base_url = f"https://github.com/orgs/{organization_name}/repositories"
    
    while True:
        url = f"{base_url}?page={page}"
        print(f"---ページ {page} ({url}) のスクレイピング開始---")

        try:
            resp = session.get(url, headers=headers, timeout=10)
            resp.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"リクエストエラー: {e}")
            break

        soup = BeautifulSoup(resp.text, 'html.parser')

        repo_links = []
        repo_links.extend(soup.find_all('a', {'data-hovercard-type': 'repository'}))
        if not repo_links:
            repo_links.extend(soup.select('h3 a'))
        if not repo_links:
            repo_links.extend(soup.select('a[itemprop="name codeRepository"]'))
        if not repo_links:
            repo_links.extend([a for a in soup.find_all('a', href=True) if f'/{organization_name}/' in a['href']])

        seen = set()
        links = []
        for a in repo_links:
            href = a.get('href') or ''
            key = href.strip() + (a.text or '').strip()
            if key and key not in seen:
                seen.add(key)
                links.append(a)

        if not links:
            print(f"ページ {page} でリポジトリリンクが見つかりませんでした。ページングを終了します。")
            break

        page_scraped_count = 0
        for link in links:
            href = link.get('href', '') or ''
            
            repo_name = ''
            if href.startswith('/'):
                parts = href.rstrip('/').split('/')
                if len(parts) >= 2 and parts[-2].lower() == organization_name.lower():
                    repo_name = parts[-1]
                else:
                    repo_name = link.text.strip()
            else:
                repo_name = link.text.strip() or href.rstrip('/').split('/')[-1]

            if not repo_name:
                 continue
            repo = link.find_parent('div', class_='Box-row') or link.find_parent('li') or link.find_parent('div')

            language = 'N/A'
            stars_text = ''
            
            if repo:
                lang_tag = repo.find('span', itemprop='programmingLanguage')
                if lang_tag and lang_tag.text:
                    language = lang_tag.text.strip()
                else:
                    alt = repo.select_one('[data-test-selector="repo-language-color"]') or repo.select_one('.f6 .mr-3')
                    if alt and alt.text:
                        language = alt.text.strip()
                
                stars_tag = repo.find('a', href=lambda h: h and h.endswith('/stargazers'))
                if not stars_tag:
                    stars_tag = repo.find('a', href=lambda h: h and 'stargazers' in h)
                if not stars_tag:
                    stars_tag = repo.select_one('a[href$="/stargazers"]')
                
                if stars_tag:
                    span = stars_tag.find('span')
                    stars_text = (span.text.strip() if span and span.text else stars_tag.text.strip())
            
            stars = convert_stars_to_int(stars_text)

            yield (repo_name, language, stars)
            print(f"   -> 取得: {repo_name:<40} | スター: {stars:,}")
            page_scraped_count += 1

        print(f"--- ページ {page} のスクレイピング完了。取得件数: {page_scraped_count} ---")
        
        page += 1

        time.sleep(1) 

print("スクレイピング開始")

try:
    repo_data = list(scrape_github_repos('google')) 
except Exception as e:
    print(f"エラーが発生しました: {e}")
    repo_data = []

print("\n DBへの挿入処理")
try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    cur.execute('DELETE FROM repositories')
    conn.commit()

    if repo_data:
        sql = "INSERT OR REPLACE INTO repositories (repo_name, language, stars) VALUES (?, ?, ?);"
        cur.executemany(sql, repo_data)
        conn.commit()
        print(f'\n**{len(repo_data)}**件のデータがDBに挿入されました')
    else:
        print('\n取得データなし')

except sqlite3.Error as e:
    print('DBエラーが発生しました:', e)
finally:
    if 'conn' in locals() and conn:
        conn.close()

print("\n保存データのSELECT表示処理")
try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    cur.execute("SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC;")
    
    results = cur.fetchall()
    
    if results:
        print(f"\n{len(results)}件の取得結果:")
        print(f"{'リポジトリ名'} | {'主要な言語'} | {'スターの数'}")
        print("-" * 75)
        for name, lang, stars in results:
            print(f"{name} | {lang} | {stars,}")
        print('\nSELECT文の実行とデータ表示が完了しました。')
    else:
        print('データが見つかりませんでした。')

except sqlite3.Error as e:
    print('DBエラーが発生しました:', e)
finally:
    if 'conn' in locals() and conn:
        conn.close()

スクレイピング開始
---ページ 1 (https://github.com/orgs/google/repositories?page=1) のスクレイピング開始---
   -> 取得: skia-buildbot                            | スター: 158
   -> 取得: perfetto                                 | スター: 5,000
   -> 取得: nsjail                                   | スター: 3,600
   -> 取得: nomulus                                  | スター: 1,800
   -> 取得: skia                                     | スター: 10,000
   -> 取得: device-infra                             | スター: 58
   -> 取得: android-cuttlefish                       | スター: 571
   -> 取得: meridian                                 | スター: 1,200
   -> 取得: angle                                    | スター: 3,800
   -> 取得: heir                                     | スター: 608
   -> 取得: gn-language-server                       | スター: 18
   -> 取得: zerocopy                                 | スター: 2,100
   -> 取得: matr                                     | スター: 9
   -> 取得: docsy                                    | スター: 2,900
   -> 取得: dawn                   